In [1]:
import numpy as np
import sys
import math
import itertools
import matplotlib.pyplot as plt  
from time import time 

# Algoritmos avaros

## 5.1 Introducción

En esta unidad nos dedicaremos casi enteramente a resolver problemas de optimización utilizando, como venimos haciendo, algoritmos y no heurísticas. Hacemos énfasis en este punto ya que la estrategia que veremos será reutilizada más adelante para diseñar heurísticas. Pero en lo que respecta a esta unidad, no nos alcanza con encontrar la solución que nos de un costo bajo ---o _score_ alto---, sino que buscamos la solución óptima, la que nos da el costo más bajo posible, o el _score_ más alto.

Los algoritmos _greedy_, o golosos, o avaros, pueden ser entendidos como un subtipo de los algoritmos de PD.
Resuelven problemas que pueden ser resueltos por algoritmos de PD, pero siguiendo una estrategia más directa lo que resulta en un algoritmo con una menor complejidad.
Los problemas que pueden ser resueltos por algoritmos golosos, como los que se resuelven por PD, tienen subestructura óptima. Pero a diferencia de PD, cada subproblema depende de la solución de un solo subproblema más pequeño. Esta crucial diferencia elimina la multiplicidad de caminos posibles, aplanando los árboles de recursión de PD y conviertiéndolos en un proceso lineal en el que se decide el camino óptimo de recursión a cada paso; a diferencia de PD que debe explorar distintas posibilidades para luego decidir el camino óptimo.

<img src="arbol_pd_greedy.png" alt="mm" style="width: 1000px;"/>

Árboles de subproblemas de un algoritmo de PD y uno _greedy_ para un problema hipótetico que tiene subestructura óptima y además puede ser resuelto por método _greedy_. 
Tal subproblema hipotético podría ser resuelto por PD, pero si se opera de la forma correcta, puede ser resuelto en forma directa, linealizando subproblemas y eliminando la superposición de los mismos.
En _greedy_ el camino óptimo para resolver un subproblema es evidente, esto lleva a un camino directo que lleva el problema inicial hasta el caso base.

<img src="ejer3.png" alt="mm" style="width: 600px;"/>

El ejercicio 3 de la guía de PD puede ayudar con la diferenciación. A la izquierda está la matriz de costos de la que debiamos partir desde `[0,0]` y llegar hasta `[n,m]` y a la derecha, la estructura de datos final que nuestro algoritmo de PD llenó con los costos de las soluciones óptimas a los subproblemas.
Luego de terminar nuestro algoritmo que hallaba estos costos, debíamos hallar la solución óptima en sí. Esto lo podíamos hacer tomando la estructura de datos donde memoizamos y haciendo unn _traceback_ desde la celda `[n,m]` hasta la celda `[0,0]` (por cierto, este procedimiento es muy similar al _traceback_ de Needleman-Wunsch).

Este algoritmo final de _traceback_ es un algoritmo greedy. Parte desde la celda `[n,m]`, que tiene un valor de `18`, se fija cuál es la celda contigua con menor costo y avanza hacia ella, sin plantearse un camino alternativo y aún así, halla la respuesta correcta. Esto es común a todos los algoritmos _greedy_. Éstos hacen la elección óptima local, toman la decisión que parece más conveniente en el momento y aún así llegan a la respuesta óptima a nivel global.

Ésto, claro, se debe a que PD ya hizo su trabajo y en vez de lidiar con una matriz de costos de cada celda, el algoritmo _greedy_ está lidiando con los costos **para llegar** a cada celda.

Si utilizaramos el algoritmo _greedy_ para la primer parte del problema, no tendríamos garantizada la respuesta correcta. En cambio, podríamos usar un algoritmo de PD para reemplazar al _greedy_ en la segunda parte del problema y si bien obtendríamos la respuesta correcta, sería con un exceso de cálculo; con un algoritmo _greedy_ hubiera bastado.

Pero es importante notar que las aproximaciones lidian con problemas que un análisis superficial tacharía de similares, pero que ahora los sabemos fundamentalmente distintos.

-------

## 5.2 _Stable Matching_

El ejemplo que veremos será el de un problema de decisión y no de optimización. 

En una realidad alternativa, las incorporaciones de los jugadores a los clubes se maneja de la siguiente manera:
\
hay `n` clubes ofreciendo una vacante para incorporador un jugador, de `m` disponibles. Cada club tiene un orden de preferencia de jugadores y a su vez, cada jugador tiene un orden de preferencia de clubes para jugar. Si se dan las siguientes condiciones:
 
 * Mismo número de clubes y jugadores (`n == m`).
 * Cada club ofrece 1 vacante.
 * Los órdenes de preferencia son completos, es decir, cada jugador ranquea a todos los clubes y lo mismo hace cada club con los jugadores.
 * No hay empates entre los rankings de clubes y jugadores
 
entonces existe un conjunto de apareamientos completo que no sea inestable.

**Apareamiento inestable**: ocurre cuando hay un par club-jugador que preferirían estar en la misma pareja entre sí, por sobre su pareja actual. Es decir, una vez hecho el apareamiento de todos los clubes con todos los jugadores, no debería ser posible hallar 2 pares club-jugador en donde uno de los jugadores prefiera jugar en el otro club y este otro club también prefiera a ese jugador, por sobre el que se le dió.

Nótese que este no es un problema de optimización. No buscamos maximizar la satisfacción de los clubes, ni de los jugadores, ni de ambos grupos en conjunto. Buscamos un apareamiento que sea estable, según la definición anterior. Además el apareamiento debe ser completo, no debe quedar jugador sin club, ni club sin jugador.

Siempre ayuda tener un ejemplo reducido. Las letras en mayúsculas son los clubes y las minúsculas, los jugadores. Las 2 tablas presentan sus órdenes de preferencia.

<img src="stable_matching_1.png" alt="mm" style="width: 300px;"/>

## 5.3 Posible solución al _Stable Matching_

Anteriormente dijimos que un algoritmo _greedy_ toma la decisión más óptima posible a medida que se le presentan las instancias de decisión. Si bien aquí no hay solución óptima, por que no hay nada optimizar, porque éste no es un problema de optimización, si hay que tomar decisiones. 

Así que eso haremos, a la hora de aparear `n` clubes y `m` jugadores, no nos plantearemos las $nm$ posibles parejas, sino que le daremos a cada club que se nos presente un jugador y pasaremos al próximo club y jugador (subproblema). También se podría hacer desde la perspectiva de los jugadores, pero si logramos un algoritmo correcto (que no genere parejas inestables en ningún caso), esa inversión de perspectiva no será relevante.

Comenzaremos iterando a lo largo de los clubes y dándole a cada club, el jugador de su preferencia, mientras éste esté disponible. Obtenemos el siguiente apareamiento:

$$
\{A ; r\} \newline
\{B ; s\} \newline
\{C ; q\} \newline
$$

Y así no hay apareamientos inestables, ya que cada club se quedó con su jugador preferido. Veamos que ocurre si invertimos la perspectiva

$$
\{q ; A\} \newline
\{r ; C\} \newline
\{s ; B\} \newline
$$

Tampoco es inestable.

Veamos un nuevo ejemplo:

<img src="stable_matching_2.png" alt="mm" style="width: 300px;"/>

Apareamos desde la perspectiva de los clubes:

$$
\{A ; q\} \newline
\{B ; r\} \newline
\{C ; s\} \newline
$$

Y aquí se rompe nuestra solución. $B$ hubiera preferido a $q$, que a su vez lo hubiera preferido antes que quedarse con $A$, el último club de su lista. Este apareamiento es inestable.


## 5.4 Algoritmo de Boston Pool y Gale-Shapley

Nuestro problema es que no fuimos suficientemente _greedy_. Tomamos la decisión avara para los clubes, pero no para los jugadores.

Un error común es suponer que porque un algoritmo es avaro y resuelve un subproblema de inmediato, no debe ser capaz de corregir esa decisión en un paso posterior.

El algoritmo avaro correcto itera los clubes como el anterior, pero si el jugador preferido ya tiene club, no pasa al siguiente jugador, sino que le pregunta al jugador si no preferiría descartar a su club anterior y formar una nueva pareja, ya que hay un club que lo prefiere a él. Y éste segundo paso es la clave para que no se formen parejas inestables.\
Naturalmente, esto deja a clubes rechazados sin jugadores, el algoritmo debe volver a iterar por los clubes restantes hasta que no haya club sin jugador y esto garantiza que no haya club ni jugador sin aparear.

Entonces, el procedimiento sería:

 * Un club $A$ se le ofrece al jugador $\alpha$ de su máxima preferencia, que no lo haya rechazado aún.
 * Ahora se pueden dar 3 casos:

```
1. El jugador α no tiene club y acepta tentativamente la oferta.
2. El jugador α tiene club, pero prefiere a A. Acepta tentativamente la oferta.
3. El jugador α tiene club, y lo prefiere a A. Rechaza la oferta de forma definitoria.
```

Es decir, los clubes ofertan de forma avara y los jugadores aceptan de forma avara. Y de nuevo, si invertimos los roles, nada va a cambiar. 

Si aplicamos este nuevo algoritmo al ejemplo anterior, obtenemos el apareamiento:

Primera pasada:

$$
\{A ; \} \newline
\{B ; q\} \newline
\{C ; s\} \newline
$$

Segunda pasada:
$$
\{A ; s\} \newline
\{B ; q\} \newline
\{C ; \} \newline
$$

Tercera pasada:
$$
\{A ; s\} \newline
\{B ; q\} \newline
\{C ; r\} \newline
$$

Pueden ustedes practicar con este nuevo ejemplo:

<img src="stable_matching_3.png" alt="mm" style="width: 300px;"/>

## 5.5 Corrección y complejidad del algoritmo de Boston Pool y Gale-Shapley

Es fundamental en los algoritmos avaros comprobar que nuestra solución llegue a una solución correcta, ya que no todo problema puede ser resuelto óptima o correctamente, por un algoritmo _greedy_ e incluso aquellos que si lo son, deben ser resueltos con la estrategia correcta. Ya vimos que la estrategia errónea puede llevar a una solución incorrecta, o en el caso de los problemas de optimización, una solución no óptima.

Comprobaremos las 2 propiedades que nuestra solución debe tener.

* Bajo este algoritmo, cada club hace $1 \leq ofertas \leq m$ ofertas y se termina con un apareamiento completo. Si no lo fuera, entonces habría un club que fue rechazado por todos los jugadores y para que eso ocurra, entonces los `m` jugadores deberían tener club, pero como `m==n`, entonces los `n` clubes también tendrían su jugador y por lo tanto no podría haber un club sin jugador (contradicción, absurdo).
* El apareamiento es estable. Dados un club y un jugador `{u, v}` que no fueron apareados, estos no preferirían estar juntos ya que si no están apareados se dió alguno de estos 2 casos:

```
1. el club u nunca ofertó al jugador v, en tal caso, el club tiene un jugador w al que prefiere por encima de v.
2. el club u hizo una oferta al jugador v, pero éste prefirió a un club x
```
Entonces no hay apareamiento inestable.

Y en la primera comprobación vimos un indicio de la complejidad de nuestro algoritmo. Si `m == n` y cada club oferta como máximo a todos los jugadores, y a cada paso hacemos una oferta, entonces el número de ofertas y, por lo tanto, la complejidad de nuestro algoritmo estarán acotados por $O(n)$

## 5.6 Implementación del algoritmo de Boston Pool y Gale-Shapley

In [37]:
N = 4
 
# This function returns true if
# woman 'w' prefers man 'm1' over man 'm'
def wPrefersM1OverM(prefer, w, m, m1):
     
    # Check if w prefers m over her
    # current engagment m1
    for i in range(N):
         
        # If m1 comes before m in list of w,
        # then w prefers her current engagement,
        # don't do anything
        if (prefer[w][i] == m1):
            return True
 
        # If m comes before m1 in w's list,
        # then free her current engagement
        # and engage her with m
        if (prefer[w][i] == m):
            return False
 
# Prints stable matching for N boys and N girls.
# Boys are numbered as 0 to N-1.
# Girls are numbered as N to 2N-1.
def stableMarriage(prefer):
     
    # Stores partner of women. This is our output
    # array that stores passing information.
    # The value of wPartner[i] indicates the partner
    # assigned to woman N+i. Note that the woman numbers
    # between N and 2*N-1. The value -1 indicates
    # that (N+i)'th woman is free
    wPartner = [-1 for i in range(N)]
 
    # An array to store availability of men.
    # If mFree[i] is false, then man 'i' is free,
    # otherwise engaged.
    mFree = [False for i in range(N)]
 
    freeCount = N
 
    # While there are free men
    while (freeCount > 0):
         
        # Pick the first free man (we could pick any)
        m = 0
        while (m < N):
            if (mFree[m] == False):
                break
            m += 1
 
        # One by one go to all women according to
        # m's preferences. Here m is the picked free man
        i = 0
        while i < N and mFree[m] == False:
            w = prefer[m][i]
 
            # The woman of preference is free,
            # w and m become partners (Note that
            # the partnership maybe changed later).
            # So we can say they are engaged not married
            if (wPartner[w - N] == -1):
                wPartner[w - N] = m
                mFree[m] = True
                freeCount -= 1
 
            else:
                 
                # If w is not free
                # Find current engagement of w
                m1 = wPartner[w - N]
 
                # If w prefers m over her current engagement m1,
                # then break the engagement between w and m1 and
                # engage m with w.
                if (wPrefersM1OverM(prefer, w, m, m1) == False):
                    wPartner[w - N] = m
                    mFree[m] = True
                    mFree[m1] = False
            i += 1
 
            # End of Else
        # End of the for loop that goes
        # to all women in m's list
    # End of main while loop
 
    # Print solution
    print("Woman ", " Man")
    for i in range(N):
        print(i + N, "\t", wPartner[i])
 
# Driver Code
prefer = [[7, 5, 6, 4], [5, 4, 6, 7],
          [4, 5, 6, 7], [4, 5, 6, 7],
          [0, 1, 2, 3], [0, 1, 2, 3],
          [0, 1, 2, 3], [0, 1, 2, 3]]
 
stableMarriage(prefer)

Woman   Man
4 	 2
5 	 1
6 	 3
7 	 0


In [8]:
orden_clubes = np.array([[7, 5, 6, 4], [5, 4, 6, 7],
          [4, 5, 6, 7], [4, 5, 6, 7]])
orden_jugadores = np.array([[0, 1, 2, 3], [0, 1, 2, 3],
          [0, 1, 2, 3], [0, 1, 2, 3]])

In [13]:
orden_jugadores[0, :]

array([0, 1, 2, 3])

In [20]:
a = np.array([4, 3, 2, 1, 0])

4

In [3]:
def 
    orden_club_viejo = np.argwhere(a == orden_jugadores[jugador, :] == club_viejo)[0][0]
    orden_club_nuevo = np.argwhere(a == orden_jugadores[jugador, :] == club_nuevo)[0][0]
    if (orden_club_nuevo < orden_club_viejo):
        as

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

For simplicity, I’ll assume from now on that there are exactly the same
number of doctors and hospitals; each hospital offers exactly one internship;
each doctor ranks all hospitals and vice versa; and finally, there are no ties in
the doctors’ or hospitals’ rankings.

Los algoritmos para problemas de optimización normalmente pasan por una secuencia de pasos,
con un conjunto de opciones en cada paso. Para muchos problemas de optimización, el uso de dinámicas
programar para determinar las mejores opciones es excesivo; al-
los goritmos servirán. Un algoritmo codicioso siempre toma la decisión que mejor se ve
el momento. Es decir, hace una elección óptima localmente con la esperanza de que esta elección
conducirá a una solución óptima a nivel mundial. Este capítulo explora los problemas de optimización
lemas para los que los algoritmos codiciosos proporcionan soluciones óptimas. Antes de leer esto
capítulo, debería leer sobre programación dinámica en el Capítulo 15, particularmente
Sección 15.3.
Los algoritmos codiciosos no siempre producen soluciones óptimas, pero para muchos problemas
ellas hacen

¿Y si pudiéramos elegir una actividad para agregar a nuestra solución óptima sin tener
resolver primero todos los subproblemas? Eso podría salvarnos de tener que considerar todo
las opciones inherentes a la recurrencia (16.2). De hecho, para el problema de selección de actividades,
necesitamos considerar solo una opción: la elección codiciosa.
¿Qué entendemos por elección codiciosa para el problema de selección de actividad? Intu-
ition sugiere que debemos elegir una actividad que deje el recurso disponible
para tantas otras actividades como sea posible. Ahora, de las actividades que terminamos eligiendo
ing, uno de ellos debe ser el primero en terminar. Nuestra intuición nos dice, por tanto,
elegir la actividad en S con la hora de finalización más temprana, ya que eso dejaría el
recurso disponible para tantas de las actividades que le siguen como sea posible. (Si mas
que una actividad en S tiene la hora de finalización más temprana, entonces podemos elegir cualquiera
actividad.) En otras palabras, dado que las actividades se ordenan en monotónicamente creciente
orden por hora de finalización, la elección codiciosa es la actividad a1. Elegir la primera actividad
terminar no es la única manera de pensar en hacer una elección codiciosa para este problema;
El ejercicio 16.1-3 le pide que explore otras posibilidades.
Si tomamos la decisión codiciosa, solo nos queda un subproblema por resolver:
encontrar actividades que comiencen después de que termine un 1. ¿Por qué no tenemos que considerar una
actividades que terminan antes de que comience a1? Tenemos que s1 <f1, y f1 es el primero
tiempo de finalización de cualquier actividad y, por lo tanto, ninguna actividad puede tener un tiempo de finalización inferior a
o igual a s1. Por lo tanto, todas las actividades que sean compatibles con la actividad a1 deben comenzar
después de que a1 termine.

Un algoritmo codicioso obtiene una solución óptima a un problema haciendo una secuencia
de opciones. En cada punto de decisión, el algoritmo toma la decisión que parece mejor en
el momento. Esta estrategia heurística no siempre produce una solución óptima,
pero como vimos en el problema de selección de actividades, a veces ocurre. Esta sección
analiza algunas de las propiedades generales de los métodos codiciosos.
El proceso que seguimos en la Sección 16.1 para desarrollar un algoritmo codicioso fue
un poco más complicado de lo habitual. Pasamos por los siguientes pasos:
1. Determine la subestructura óptima del problema.
2. Desarrolle una solución recursiva. (Para el problema de selección de actividad, formulamos
recurrencia (16.2), pero pasamos por alto el desarrollo de un algoritmo recursivo basado
en esta recurrencia.)
3. Demuestre que si tomamos la decisión codiciosa, sólo queda un subproblema.
4. Demuestre que siempre es seguro tomar una decisión codiciosa. (Los pasos 3 y 4 pueden ocurrir
en cualquier orden.)
5. Desarrolle un algoritmo recursivo que implemente la estrategia codiciosa.
6. Convierta el algoritmo recursivo en un algoritmo iterativo.
Al seguir estos pasos, vimos con gran detalle la programación dinámica
derpinnings de un algoritmo codicioso. Por ejemplo, en el problema de selección de actividades,
Primero definimos los subproblemas Sij, donde tanto i como j variaban. Entonces encontramos
que si siempre tomáramos la decisión codiciosa, podríamos restringir los subproblemas para que
de la forma Sk.
Alternativamente, podríamos haber diseñado nuestra subestructura óptima con un codicioso
elección en mente, de modo que la elección deje solo un subproblema por resolver. En el
problema de selección de actividad, podríamos haber comenzado eliminando el segundo subíndice
y definir subproblemas de la forma Sk. Entonces, podríamos haber demostrado que un codicioso
elección (la primera actividad que debo terminar en Sk), combinada con una solución óptima para
el conjunto restante Sm de actividades compatibles, produce una solución óptima para Sk.
De manera más general, diseñamos algoritmos codiciosos de acuerdo con la siguiente secuencia
de pasos:

1. Considere el problema de optimización como uno en el que tomamos una decisión y nos quedamos
con un subproblema que resolver.
2. Demuestre que siempre hay una solución óptima al problema original que hace
la elección codiciosa, de modo que la elección codiciosa sea siempre segura.
3. Demuestre la subestructura óptima mostrando que, habiendo hecho que los codiciosos
elección, lo que queda es un subproblema con la propiedad de que si combinamos un
solución óptima al subproblema con la elección codiciosa que hemos hecho,
llegar a una solución óptima al problema original.
Usaremos este proceso más directo en secciones posteriores de este capítulo. Sin embargo-
menos, debajo de cada algoritmo codicioso, casi siempre hay un más engorroso
solución de programación dinámica.
¿Cómo podemos saber si un algoritmo codicioso resolverá una optimización en particular?
¿problema? De ninguna manera funciona todo el tiempo, pero la propiedad de elección codiciosa y óptima
La subestructura son los dos ingredientes clave. Si podemos demostrar que el problema
tiene estas propiedades, entonces estamos en camino de desarrollar un algoritmo codicioso
para ello.

### Greedy choice property

El primer ingrediente clave es la propiedad de elección codiciosa: podemos ensamblar una
solución óptima tomando decisiones localmente óptimas (codiciosas). En otras palabras, cuando
estamos considerando qué elección tomar, tomamos la decisión que se ve mejor en
el problema actual, sin considerar los resultados de los subproblemas.
Aquí es donde los algoritmos codiciosos se diferencian de la programación dinámica. En dinámica
programación, hacemos una elección en cada paso, pero la elección generalmente depende de la
soluciones a subproblemas. En consecuencia, normalmente resolvemos la programación dinámica
problemas de abajo hacia arriba, progresando de subproblemas más pequeños a más grandes
subproblemas. (Alternativamente, podemos resolverlos de arriba hacia abajo, pero memorizando.
Por supuesto, aunque el código funciona de arriba hacia abajo, todavía debemos resolver el subproblema
lemas antes de tomar una decisión.) En un algoritmo codicioso, hacemos cualquier elección
parece mejor en este momento y luego resolver el subproblema que queda. La elección
hecho por un algoritmo codicioso puede depender de las elecciones hasta ahora, pero no puede depender de
las opciones futuras o las soluciones a los subproblemas. Por tanto, a diferencia del pro-
gramática, que resuelve los subproblemas antes de hacer la primera elección, un codicioso
El algoritmo hace su primera elección antes de resolver cualquier subproblema. Una dinámica
El algoritmo de programación procede de abajo hacia arriba, mientras que una estrategia codiciosa generalmente
progresa de arriba hacia abajo, haciendo una elección codiciosa tras otra, reduciendo
trasladar cada instancia de problema dada a una más pequeña.
Por supuesto, debemos demostrar que una elección codiciosa en cada paso produce una
solucion optima. Normalmente, como en el caso del teorema 16.1, la demostración examina
una solución globalmente óptima para algún subproblema. Luego muestra cómo modificar
la solución para sustituir la elección codiciosa por otra opción, lo que resulta en una
subproblema similar, pero menor.
Por lo general, podemos tomar la decisión codiciosa de manera más eficiente que cuando tenemos que hacerlo.
considere un conjunto más amplio de opciones. Por ejemplo, en el problema de selección de actividades, suponiendo que ya habíamos ordenado las actividades en un orden monótonamente creciente
de los tiempos de finalización, necesitábamos examinar cada actividad una sola vez. Al preprocesar el
de entrada o mediante el uso de una estructura de datos adecuada (a menudo una cola de prioridad), a menudo
puede tomar decisiones codiciosas rápidamente, produciendo así un algoritmo eficiente.

### subestructura óptima

Un problema exhibe una subestructura óptima si una solución óptima al problema
contiene en su interior soluciones óptimas a subproblemas. Esta propiedad es clave en
grediente de evaluar la aplicabilidad de la programación dinámica, así como codiciosos
algoritmos. Como ejemplo de subestructura óptima, recuerde cómo demostramos en
Sección 16.1 que si una solución óptima al subproblema Sij incluye una actividad ak,
entonces también debe contener soluciones óptimas a los subproblemas Si k y Skj. Dado
esta subestructura óptima, argumentamos que si supiéramos qué actividad usar como ak,
podría construir una solución óptima para Sij seleccionando ak junto con todas las actividades
en soluciones óptimas a los subproblemas Si k y Skj. Basado en esta observación de
subestructura óptima, pudimos idear la recurrencia (16.2) que describió
el valor de una solución óptima.
Usualmente usamos un enfoque más directo con respecto a la subestructura óptima cuando
aplicándolo a algoritmos codiciosos. Como se mencionó anteriormente, tenemos el lujo de
asumiendo que llegamos a un subproblema al haber tomado la codiciosa elección en
el problema original. Todo lo que realmente necesitamos hacer es argumentar que una solución óptima para
el subproblema, combinado con la elección codiciosa ya hecha, produce un óptimo
solución al problema original. Este esquema utiliza implícitamente la inducción en el
subproblemas para demostrar que tomar la decisión codiciosa en cada paso produce una
solucion optima

### greedy vs Programación Dinámica

Debido a que tanto la estrategia codiciosa como la de programación dinámica explotan sub-
estructura, puede tener la tentación de generar una solución de programación dinámica para una
problema cuando una solución codiciosa es suficiente o, por el contrario, podría pensar erróneamente
que una solución codiciosa funciona cuando en realidad una solución de programación dinámica se
quired. Para ilustrar las sutilezas entre las dos técnicas, investiguemos
dos variantes de un problema de optimización clásico.
El problema de la mochila 0-1 es el siguiente. Un ladrón robando una tienda encuentra n
elementos. El i-ésimo artículo vale i dólares y pesa wi libras, donde i y wi son
enteros. El ladrón quiere llevar una carga lo más valiosa posible, pero puede llevarla a
la mayor parte de W libras en su mochila, para algunos W enteros. ¿Qué artículos debería llevarse?
(A esto lo llamamos el problema de la mochila 0-1 porque para cada artículo, el ladrón debe

tómalo o déjalo atrás; no puede tomar una cantidad fraccionaria de un artículo o tomar una
artículo más de una vez.)
En el problema de la mochila fraccionada, la configuración es la misma, pero el ladrón puede tomar
fracciones de elementos, en lugar de tener que hacer una elección binaria (0-1) para cada elemento.
Puedes pensar en un artículo en el problema de la mochila 0-1 como si fuera un lingote de oro.
y un artículo en el problema de la mochila fraccionada como más bien polvo de oro.
Ambos problemas de mochila exhiben la propiedad de subestructura óptima. Por el 0-1
problema, considere la carga más valiosa que pesa como máximo W libras. Si nosotros
quitar el artículo j de esta carga, la carga restante debe ser la carga más valiosa
con un peso máximo de W w j que el ladrón puede tomar de los n 1 artículos originales
excluyendo j. Para el problema fraccional comparable, considere que si eliminamos
un peso w de un artículo j de la carga óptima, la carga restante debe ser la
carga más valiosa con un peso máximo de W w que el ladrón puede tomar del n 1
artículos originales más w j w libras del artículo j.
Aunque los problemas son similares, podemos resolver el problema de la mochila fraccionada
con una estrategia codiciosa, pero no podemos resolver el problema de 0-1 con esa estrategia. Para
resolver el problema fraccional, primero calculamos el valor por libra i = wi para cada
artículo. Obedeciendo una estrategia codiciosa, el ladrón comienza tomando la mayor cantidad posible de
el artículo con el mayor valor por libra. Si se agota el suministro de ese artículo
y todavía puede llevar más, toma la mayor cantidad posible del artículo con el siguiente
mayor valor por libra, y así sucesivamente, hasta que alcance su límite de peso W. Por lo tanto,
al ordenar los artículos por valor por libra, el algoritmo codicioso se ejecuta en O.n lg n /
tiempo. Dejamos la prueba de que el problema de la mochila fraccionada tiene la codicia
propiedad de elección como el ejercicio 16.2-1.
Para ver que esta estrategia codiciosa no funciona para el problema de la mochila 0-1,
considere el caso de problema ilustrado en la figura 16.2 (a). Este ejemplo tiene 3
artículos y una mochila que puede contener 50 libras. El artículo 1 pesa 10 libras y
vale 60 dolares. El artículo 2 pesa 20 libras y vale 100 dólares. Ítem ​​3
pesa 30 libras y vale 120 dólares. Por lo tanto, el valor por libra del artículo 1 es
6 dólares por libra, que es mayor que el valor por libra de cualquier artículo 2 (5
dólares por libra) o el artículo 3 (4 dólares por libra). La estrategia codiciosa, por lo tanto,
tomaría el artículo 1 primero. Como puede ver en el análisis de caso de la figura 16.2 (b),
sin embargo, la solución óptima toma los elementos 2 y 3, dejando atrás el elemento 1. Los dos
las posibles soluciones que toman el elemento 1 son ambas subóptimas.
Para el problema fraccionario comparable, sin embargo, la estrategia codiciosa, que
toma el elemento 1 primero, produce una solución óptima, como se muestra en la figura 16.2 (c). Tak-
El ítem 1 no funciona en el problema 0-1 porque el ladrón no puede llenar su
mochila a capacidad, y el espacio vacío reduce el valor efectivo por libra de
su carga. En el problema 0-1, cuando consideramos si incluir un elemento en el
mochila, debemos comparar la solución con el subproblema que incluye el artículo
con la solución al subproblema que excluye el artículo antes de que podamos hacer el
elección. El problema formulado de esta manera da lugar a muchos sub-
problemas, un sello distintivo de la programación dinámica y, de hecho, como el ejercicio 16.2-2
le pide que muestre, podemos usar programación dinámica para resolver el problema 0-1.

-------------

--------------

## 4.3 

In [5]:
def torre_de_hanoi(n, fuente, destino, temporal):
    if n > 0
        torre_de_hanoi(n − 1, fuente, temporal, destino)
        muevo_disco(n, fuente, destino)
        torre_de_hanoi(n − 1, temporal, destino, fuente)
    return

$$ T(n) = 2 T(n-1) + 1 $$

-------

## 4.4 

-------------------

## 4.6 

----------------

## 4.7 Conclusión

* 
* Leer del Cormen ; Chapter 16: Greedy Algorithms: p(414-436)

--------------------

### Contenidos a explicar durante la práctica

1. 